In [8]:
from NMR_predict import GPR_NMR

In [12]:
# Define path to xyz files and path to SOAP files
XYZ_directory = '/home/alex/Pt_NMR/data/structures/'
SOAP_directory = '/home/alex/Pt_NMR/data/representations/SOAP/'

# Define list of SOAP params [r_cut, n_max, l_max]
params = [2.0, 1, 10]

# Create instance of GPR_NMR class
pt_gpr = GPR_NMR(descriptor_path=SOAP_directory, descriptor_type='SOAP',
                 descriptor_params=params, central_atom='Pt', mode ='write', xyz_path=XYZ_directory, xyz_base='st_')


In [13]:
# Define path and column name of the csv where labels (targets) are stored
target_name = 'Experimental'
target_path = '/home/alex/Pt_NMR/data/labels/final_data_corrected'

# Use predict function for GPR and obtaining CV prediction errors
errors_std = pt_gpr.predict(regressor_type='GPR', kernel_degree=2,
                            target_path=target_path, target_name=target_name,
                            normalize=False, alpha=0.1)

print(errors_std)

Species present in dataset: ['P', 'S', 'Cl', 'N', 'As', 'Sb', 'O', 'I', 'Se', 'Si', 'F', 'H', 'Te', 'Br', 'Sn', 'Pt', 'Ge', 'C']
RMSE (4-fold CV): 217.53377589064817 [ppm] 45.899976196270295 [ppm] (STDEV)
MAE (4-fold CV): 119.70423901934981 [ppm] 19.671877159762207 [ppm] (STDEV)
(119.70423901934981, 19.671877159762207, 217.53377589064817, 45.899976196270295)


In [14]:
XYZ_directory = '/home/alex/Pt_NMR/data/structures/'
SOAP_directory = '/home/alex/Pt_NMR/data/representations/APE_RF/'

# Define list of SOAP params [r_cut, n_max, l_max]
params = [3.0, 1000]

# Create instance of GPR_NMR class
pt_gpr = GPR_NMR(descriptor_path=SOAP_directory, descriptor_type='APE_RF',
                 descriptor_params=params, central_atom='Pt', mode ='write', xyz_path=XYZ_directory, xyz_base='st_')

In [15]:
# Define path and column name of the csv where labels (targets) are stored

target_name = 'Experimental'
target_path = '/home/alex/Pt_NMR/data/labels/final_data_corrected'

# Use predict function for regression and obtaining CV prediction errors

errors_std = pt_gpr.predict(regressor_type='GPR', kernel_degree=2,
                            target_path=target_path, target_name=target_name,
                            normalize=False, alpha=10.0)

print(errors_std)

RMSE (4-fold CV): 207.41602852454292 [ppm] 13.077343231011563 [ppm] (STDEV)
MAE (4-fold CV): 147.5007397240991 [ppm] 8.124650178814962 [ppm] (STDEV)
(147.5007397240991, 8.124650178814962, 207.41602852454292, 13.077343231011563)
